# Installs

In [1]:
!pip install pyarrow

# Imports

In [4]:
import pyarrow

In [8]:
import numpy as np

In [2]:
import pandas as pd

In [6]:
import sklearn

In [9]:
from sklearn.feature_extraction import DictVectorizer

# Download Data

In [10]:
df_jan23 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [11]:
df_feb23 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [12]:
df_jan23.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


# Question 1: No. of columns in Jan23 dataset

In [13]:
len(df_jan23.columns)

19

# Question 2: Calculate Duration

In [14]:
# Convert columns to datetime

df_jan23['tpep_pickup_datetime'] = pd.to_datetime(df_jan23['tpep_pickup_datetime'])
df_jan23['tpep_dropoff_datetime'] = pd.to_datetime(df_jan23['tpep_dropoff_datetime'])

In [15]:
df_feb23['tpep_pickup_datetime'] = pd.to_datetime(df_feb23['tpep_pickup_datetime'])
df_feb23['tpep_dropoff_datetime'] = pd.to_datetime(df_feb23['tpep_dropoff_datetime'])

In [16]:
# Subtract the columns to get a trip_duration column

df_jan23['trip_duration'] = df_jan23['tpep_dropoff_datetime'] - df_jan23['tpep_pickup_datetime']
df_feb23['trip_duration'] = df_feb23['tpep_dropoff_datetime'] - df_feb23['tpep_pickup_datetime']

In [19]:
# Convert the trip_duration column to hourly basis
df_jan23['trip_duration_mins']  = df_jan23['trip_duration'] / pd.Timedelta(minutes=1)
df_feb23['trip_duration_mins']  = df_feb23['trip_duration'] / pd.Timedelta(minutes=1)

In [20]:
# Calculate the standard deviation of the trip_duration column

jan23_std_dev_trip_duration_mins = df_jan23['trip_duration_mins'].std()

feb23_std_dev_trip_duration_mins = df_feb23['trip_duration_mins'].std()

In [21]:
print("Jan 2023", round(jan23_std_dev_trip_duration_mins,2))

Jan 2023 42.59


In [22]:
print("Feb 2023", round(feb23_std_dev_trip_duration_mins,2))

Feb 2023 42.84


# Question 3: Remove Outliers

In [27]:
jan23_total_records_before = df_jan23.shape[0]
print(jan23_total_records_before)

3066766


In [28]:
feb23_total_records_before = df_feb23.shape[0]
print(feb23_total_records_before)

2913955


In [29]:
# Filter to keep durations between 1 and 60 minutes (inclusive)
filtered_df_jan23 = df_jan23[(df_jan23['trip_duration_mins'] >= 1) & (df_jan23['trip_duration_mins'] <= 60)].copy()

In [30]:
# Filter to keep durations between 1 and 60 minutes (inclusive)
filtered_df_feb23 = df_feb23[(df_feb23['trip_duration_mins'] >= 1) & (df_feb23['trip_duration_mins'] <= 60)].copy()

In [31]:
jan23_total_records_after  = filtered_df_jan23.shape[0]
print(jan23_total_records_after)

3009173


In [32]:
feb23_total_records_after  = filtered_df_feb23.shape[0]
print(feb23_total_records_after)

2855951


In [33]:
# Calculate the fraction of records left
jan23_fraction_remaining = jan23_total_records_after / jan23_total_records_before

In [34]:
# Calculate the fraction of records left
feb23_fraction_remaining = feb23_total_records_after / feb23_total_records_before

In [35]:
print(round(jan23_fraction_remaining*100,0))

98.0


In [36]:
print(round(feb23_fraction_remaining*100,0))

98.0


# Question 4: One-Hot Encoding

In [37]:
# Convert pickup and dropoff location IDs to strings
filtered_df_jan23['PULocationID'] = filtered_df_jan23['PULocationID'].astype(str)
filtered_df_jan23['DOLocationID'] = filtered_df_jan23['DOLocationID'].astype(str)

In [38]:
# Convert pickup and dropoff location IDs to strings
filtered_df_feb23['PULocationID'] = filtered_df_feb23['PULocationID'].astype(str)
filtered_df_feb23['DOLocationID'] = filtered_df_feb23['DOLocationID'].astype(str)

In [39]:
# Turn the DataFrame into a list of dictionaries
jan23_data_dict = filtered_df_jan23[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [40]:
# Turn the DataFrame into a list of dictionaries
feb23_data_dict = filtered_df_feb23[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [66]:
# Fit a DictVectorizer
dv = DictVectorizer()

In [67]:
jan23_feature_matrix = dv.fit_transform(jan23_data_dict)

In [69]:
print(jan23_feature_matrix[:5])

  (0, 43)	1.0
  (0, 325)	1.0
  (1, 148)	1.0
  (1, 456)	1.0
  (2, 149)	1.0
  (2, 461)	1.0
  (3, 227)	1.0
  (3, 299)	1.0
  (4, 237)	1.0
  (4, 266)	1.0


In [93]:
print(feb23_feature_matrix[:5])

  (0, 67)	1.0
  (0, 304)	1.0
  (1, 173)	1.0
  (1, 293)	1.0
  (2, 47)	1.0
  (2, 325)	1.0
  (3, 147)	1.0
  (3, 310)	1.0
  (4, 156)	1.0
  (4, 298)	1.0


In [71]:
# Assuming 'feature_matrix' is the matrix obtained after fitting the DictVectorizer
num_rows_jan23, num_columns_jan23 = jan23_feature_matrix.shape

print("Number of rows Jan 2023:", num_rows_jan23)
print("Number of columns Jan 2023:", num_columns_jan23)


Number of rows Jan 2023: 3009173
Number of columns Jan 2023: 515


In [72]:
# Assuming 'feature_matrix' is the matrix obtained after fitting the DictVectorizer
num_rows_feb23, num_columns_feb23 = feb23_feature_matrix.shape

print("Number of rows Feb 2023:", num_rows_feb23)
print("Number of columns Feb 2023:", num_columns_feb23)


Number of rows Feb 2023: 2855951
Number of columns Feb 2023: 515


# Question 5: Model Training

In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [75]:
# Create and fit the linear regression model
model = LinearRegression()
model.fit(jan23_feature_matrix, filtered_df_jan23['trip_duration_mins'])

LinearRegression()

In [77]:
# Predict duration using the trained model
jan23_predicted_duration = model.predict(jan23_feature_matrix)

In [79]:
# Calculate the RMSE on the training data
rmse_train_jan23 = np.sqrt(mean_squared_error(filtered_df_jan23['trip_duration_mins'], jan23_predicted_duration))

In [87]:
print("RMSE on train:", rmse_train_jan23)

RMSE on train: 7.649261027855596


# Quesion 6: Model Evaluation

In [ ]:
# Transform the validation dataset into the same feature representation
feature_matrix_validation = dv.transform(feb23_data_dict)

In [84]:
# Predict duration for the validation dataset
predicted_duration_validation = model.predict(feature_matrix_validation)

In [89]:
# Calculate the RMSE on the validation data
rmse_validation = np.sqrt(mean_squared_error(filtered_df_feb23['trip_duration_mins'], predicted_duration_validation))

In [92]:
print("RMSE on validation:", rmse_validation)

RMSE on validation: 7.8118325109416915
